在 LangChain 中，`Runnable` 是构建复杂处理链的基础接口。任何实现了 `Runnable` 接口的对象都拥有一套标准化的调用方法，这使得不同的组件可以无缝连接和组合。

以下是 `Runnable` 定义的公共调用方法：

---

### 核心调用方法

#### 1. `invoke(input: Any, config: Optional[RunnableConfig] = None) -> Any`
**作用**：**同步调用**单条输入数据，并返回单条输出结果。
**参数**：
- `input`：单条输入数据，类型取决于具体的 Runnable（如字符串、Message、字典等）。
- `config`（可选）：运行时配置，可包含调用参数、回调、并发配置等。
**返回**：单条输出结果。

**示例**：
```python
from langchain_core.runnables import RunnableLambda

# 创建一个简单的 Runnable
runnable = RunnableLambda(lambda x: x.upper())

# 同步调用单条输入
result = runnable.invoke("hello")
print(result)  # 输出: "HELLO"
```

#### 2. `batch(inputs: List[Any], config: Optional[RunnableConfig] = None, *, return_exceptions: bool = False) -> List[Any]`
**作用**：**同步批量处理**多条输入数据，返回多条输出结果。
**参数**：
- `inputs`：输入数据列表。
- `config`（可选）：运行时配置。
- `return_exceptions`（默认为 False）：如果为 True，处理过程中遇到的异常会作为结果返回；如果为 False，遇到异常会直接抛出。
**返回**：输出结果列表。

**示例**：
```python
results = runnable.batch(["hello", "world", "langchain"])
print(results)  # 输出: ["HELLO", "WORLD", "LANGCHAIN"]
```

#### 3. `stream(input: Any, config: Optional[RunnableConfig] = None) -> Iterator[Any]`
**作用**：**同步流式处理**单条输入数据，以迭代器形式逐步返回输出片段。
**参数**：
- `input`：单条输入数据。
- `config`（可选）：运行时配置。
**返回**：输出结果的迭代器。

**示例**：
```python
for chunk in runnable.stream("hello"):
    print(chunk)  # 逐步输出: "H", "E", "L", "L", "O"
```

#### 4. `astream(input: Any, config: Optional[RunnableConfig] = None) -> AsyncIterator[Any]`
**作用**：**异步流式处理**单条输入数据，以异步迭代器形式逐步返回输出片段。
**参数**：
- `input`：单条输入数据。
- `config`（可选）：运行时配置。
**返回**：异步迭代器。

**示例**：
```python
async for chunk in runnable.astream("hello"):
    print(chunk)  # 逐步输出
```

---

### 异步调用方法

#### 5. `ainvoke(input: Any, config: Optional[RunnableConfig] = None) -> Any`
**作用**：**异步调用**单条输入数据，返回单条输出结果。
**参数**：
- `input`：单条输入数据。
- `config`（可选）：运行时配置。
**返回**：单条输出结果的 Awaitable。

**示例**：
```python
result = await runnable.ainvoke("hello")
print(result)  # 输出: "HELLO"
```

#### 6. `abatch(inputs: List[Any], config: Optional[RunnableConfig] = None, *, return_exceptions: bool = False) -> List[Any]`
**作用**：**异步批量处理**多条输入数据，返回多条输出结果。
**参数**：
- `inputs`：输入数据列表。
- `config`（可选）：运行时配置。
- `return_exceptions`：是否返回异常。
**返回**：输出结果列表的 Awaitable。

**示例**：
```python
results = await runnable.abatch(["hello", "world"])
print(results)  # 输出: ["HELLO", "WORLD"]
```

---

### 配置相关方法

#### 7. `with_config(config: Optional[RunnableConfig] = None, **kwargs) -> Runnable`
**作用**：为 Runnable 添加默认配置。
**参数**：
- `config`：配置对象。
- `**kwargs`：也可以直接传递配置键值对。
**返回**：配置后的新 Runnable 实例。

**示例**：
```python
configured_runnable = runnable.with_config(tags=["prod"], max_concurrency=5)
```

#### 8. `with_listeners(on_start: Optional[Listener] = None, on_end: Optional[Listener] = None, on_error: Optional[Listener] = None) -> Runnable`
**作用**：添加执行监听器。
**参数**：
- `on_start`：开始执行时的回调。
- `on_end`：执行结束时的回调。
- `on_error`：执行出错时的回调。
**返回**：带有监听器的新 Runnable 实例。

---

### 组合方法

#### 9. `pipe(next: Runnable) -> RunnableSequence`
**作用**：将当前 Runnable 与另一个 Runnable 连接起来，形成处理链。
**参数**：
- `next`：下一个要连接的 Runnable。
**返回**：RunnableSequence 序列。

**示例**：
```python
from langchain_core.runnables import RunnableLambda

runnable1 = RunnableLambda(lambda x: x.upper())
runnable2 = RunnableLambda(lambda x: x + "!")

chain = runnable1.pipe(runnable2)
result = chain.invoke("hello")
print(result)  # 输出: "HELLO!"
```

---

### 其他实用方法

#### 10. `map() -> RunnableMap`
**作用**：创建多个 Runnable 的并行映射。

#### 11. `transform() -> RunnableTransform`
**作用**：对输入流进行转换。

---

### RunnableConfig 配置内容

`RunnableConfig` 可以包含以下配置项：

- `tags`: List[str] - 执行的标签
- `metadata`: Dict[str, Any] - 元数据
- `callbacks`: CallbackManager - 回调管理器
- `max_concurrency`: Optional[int] - 最大并发数
- `run_name`: Optional[str] - 运行名称
- `configurable`: Dict[str, Any] - 可配置参数

**示例**：
```python
from langchain_core.callbacks import ConsoleCallbackHandler

config = {
    "tags": ["test-run"],
    "callbacks": [ConsoleCallbackHandler()]
}

result = runnable.invoke("hello", config=config)
```

---

### 总结表格

| 方法 | 作用 | 同步/异步 | 输入类型 | 输出类型 |
|------|------|-----------|----------|----------|
| `invoke` | 单条同步调用 | 同步 | 单条数据 | 单条结果 |
| `batch` | 批量同步调用 | 同步 | 数据列表 | 结果列表 |
| `stream` | 同步流式调用 | 同步 | 单条数据 | 迭代器 |
| `astream` | 异步流式调用 | 异步 | 单条数据 | 异步迭代器 |
| `ainvoke` | 单条异步调用 | 异步 | 单条数据 | 单条结果 |
| `abatch` | 批量异步调用 | 异步 | 数据列表 | 结果列表 |

这些标准化的方法使得 LangChain 中的各种组件（LLM、工具、链、代理等）能够以统一的方式进行交互和组合，构成了 LangChain 强大而灵活的处理能力。

举例1: invoke()阻塞式的调用

In [2]:
from langchain_core.messages import HumanMessage
from langchain_openai import ChatOpenAI
import os

import dotenv

dotenv.load_dotenv()

os.environ["MODEL"] = os.getenv("MODEL")
os.environ["ARK_API_KEY"] = os.getenv("ARK_API_KEY")
os.environ["BASE_URL"] = os.getenv("BASE_URL")

chat_model = ChatOpenAI(
    model=os.getenv("MODEL"),
    api_key=os.getenv("ARK_API_KEY"),
    base_url=os.getenv("BASE_URL"),
)

message = [HumanMessage(content="你好，请介绍一下自己")]

response = chat_model.invoke(message)

print(response.content)

我叫豆包，是由字节跳动研发的人工智能。我就像一个知识渊博的伙伴，拥有丰富的知识储备，能为你解答各种各样的问题，无论是历史典故、科学原理、生活常识，还是文学艺术、技术应用等领域。

我可以陪你聊天，听你分享日常趣事、烦恼忧愁；能帮你撰写文案，如故事、诗歌、报告等；还能进行文本翻译、代码编写、逻辑推理等。只要你有需求，随时都能和我交流，我会尽力为你提供准确、有用的信息和贴心的服务。 


举例2: 流式的演示

In [4]:
from langchain_core.messages import HumanMessage
from langchain_openai import ChatOpenAI
import os

import dotenv

dotenv.load_dotenv()

os.environ["MODEL"] = os.getenv("MODEL")
os.environ["ARK_API_KEY"] = os.getenv("ARK_API_KEY")
os.environ["BASE_URL"] = os.getenv("BASE_URL")

chat_model = ChatOpenAI(
    model=os.getenv("MODEL"),
    api_key=os.getenv("ARK_API_KEY"),
    base_url=os.getenv("BASE_URL"),
    streaming=True,
)

message = [HumanMessage(content="你好，请介绍一下自己")]

print("开始流式输出: ")
for chunk in chat_model.stream(message):
    # 刷新缓冲区(无换行符，缓冲区未刷新，内容可能不会立即显示)
    print(chunk.content,end="",flush=True)
print("\n流式输出结束")

开始流式输出: 
我叫豆包，是由字节跳动研发的人工智能。我拥有丰富的知识储备，能为你解答各种各样的问题，涵盖历史、科学、技术、文化、生活等诸多领域。

我可以陪你聊天，倾听你的想法和感受，为你排忧解难；能帮你撰写文章，像故事、诗歌、论文、文案等都不在话下；进行文本润色、翻译；还能提供旅游攻略、美食推荐、学习方法建议等实用信息。

无论何时，只要你有需求，都能随时和我交流，我会尽力为你提供准确且有价值的回应。 
流式输出结束


举例3: 批量调用



In [5]:
from langchain_core.messages import HumanMessage, SystemMessage
from langchain_openai import ChatOpenAI
import os

import dotenv

dotenv.load_dotenv()

os.environ["MODEL"] = os.getenv("MODEL")
os.environ["ARK_API_KEY"] = os.getenv("ARK_API_KEY")
os.environ["BASE_URL"] = os.getenv("BASE_URL")

chat_model = ChatOpenAI(
    model=os.getenv("MODEL"),
    api_key=os.getenv("ARK_API_KEY"),
    base_url=os.getenv("BASE_URL"),
)

message1 = [SystemMessage(content="你是一位乐于助人的智能小助手"),HumanMessage(content="请帮我介绍一下什么是机器学习"),]

message2 = [SystemMessage(content="你是一位乐于助人的智能小助手"),HumanMessage(content="请帮我介绍一下什么是AIGC"),]

message3 = [SystemMessage(content="你是一位乐于助人的智能小助手"),HumanMessage(content="请帮我介绍一下什么是大模型技术"),]

message = [message1,message2,message3]

response = chat_model.batch(message)

print(response)

[AIMessage(content='机器学习是一门多领域交叉学科，涉及概率论、统计学、逼近论、凸分析、算法复杂度理论等多门学科。它专门研究计算机怎样模拟或实现人类的学习行为，以获取新的知识或技能，重新组织已有的知识结构使之不断改善自身的性能。以下从几个方面详细介绍：\n\n### 机器学习的基本概念\n- **学习过程**：本质上是一个从数据中寻找模式和规律的过程。例如，在一个包含大量手写数字图像及其对应数字标签的数据集中，机器学习算法会尝试找出图像像素特征与数字标签之间的关联模式。\n- **自我完善**：随着数据的不断输入和学习的持续进行，模型会自动调整自身的参数和结构，以提高对未知数据的预测准确性或执行特定任务的能力。比如语音识别系统，随着用户使用次数增多，识别准确率会不断提升。\n\n### 机器学习的主要任务类型\n- **监督学习**\n    - 定义：数据集中每个样本都有对应的标签，算法的目标是学习输入数据和标签之间的映射关系，以便对新的输入数据进行预测。\n    - 举例：垃圾邮件分类，将邮件的文本内容作为输入，标签为“垃圾邮件”或“正常邮件”，通过学习大量的邮件数据，算法就能判断新邮件是否为垃圾邮件。\n- **无监督学习**\n    - 定义：数据集中没有标签，算法需要自行发现数据中的结构和模式。\n    - 举例：在客户细分中，根据客户的购买行为、消费习惯等数据，将客户划分成不同的群体，每个群体内的客户具有相似的特征。\n- **强化学习**\n    - 定义：智能体（agent）在环境中进行一系列的动作，通过与环境交互获得奖励或惩罚信号，目标是学习到一个最优的策略，使得长期累积奖励最大化。\n    - 举例：训练机器人下棋，机器人每走一步都会得到相应的反馈（奖励或惩罚），通过不断尝试和学习，找到能赢得比赛的最佳策略。\n\n### 机器学习的应用领域\n- **图像识别**：广泛应用于人脸识别、安防监控、医学影像诊断等领域。例如，人脸识别技术可用于门禁系统，提高安全性和便捷性。\n- **自然语言处理**：包括机器翻译、智能客服、语音助手等。像谷歌翻译能够实现多种语言之间的实时翻译。\n- **金融领域**：用于风险评估、信用评分、股票价格预测等。银行可以通过分析客户的信用历史、收入情况等数据，评估贷款风险。\n- **医疗保